<a href="https://colab.research.google.com/github/neogeolee/Deeplearning/blob/master/Chatbot_create_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 날짜 : 2020/09/10
# 이름 : 이태훈
# 내용 : LSTM 챗봇 실습하기
# 출처 : https://wikidocs.net/45101

import codecs
import tensorflow as tf
import keras
import numpy as np
import random, sys
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Activation, Dropout, Embedding
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.models import load_model
from bs4 import BeautifulSoup

In [ ]:
# 파일 로드 함수정의
def load_data(file):
  result = []

  with open(file, 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()

    for line in lines:
      data = line.split(',')
      data = data[0] + ' ' + data[1] + '\n'
      result.append(data)

  result = result[1:] # header정보 제외
  return result

In [ ]:
# 데이터 로드
chat_dataset = load_data('/content/drive/My Drive/Tensorflow_works/data/ChatbotData.csv')
text = ' '.join(chat_dataset)
print(text)

12시 땡! 하루가 또 가네요.
 1지망 학교 떨어졌어 위로해 드립니다.
 3박4일 놀러가고 싶다 여행은 언제나 좋죠.
 3박4일 정도 놀러가고 싶다 여행은 언제나 좋죠.
 PPL 심하네 눈살이 찌푸려지죠.
 SD카드 망가졌어 다시 새로 사는 게 마음 편해요.
 SD카드 안돼 다시 새로 사는 게 마음 편해요.
 SNS 맞팔 왜 안하지ㅠㅠ 잘 모르고 있을 수도 있어요.
 SNS 시간낭비인 거 아는데 매일 하는 중 시간을 정하고 해보세요.
 SNS 시간낭비인데 자꾸 보게됨 시간을 정하고 해보세요.
 SNS보면 나만 빼고 다 행복해보여 자랑하는 자리니까요.
 가끔 궁금해 그 사람도 그럴 거예요.
 가끔 뭐하는지 궁금해 그 사람도 그럴 거예요.
 가끔은 혼자인게 좋다 혼자를 즐기세요.
 가난한 자의 설움 돈은 다시 들어올 거예요.
 가만 있어도 땀난다 땀을 식혀주세요.
 가상화폐 쫄딱 망함 어서 잊고 새출발 하세요.
 가스불 켜고 나갔어 빨리 집에 돌아가서 끄고 나오세요.
 가스불 켜놓고 나온거 같아 빨리 집에 돌아가서 끄고 나오세요.
 가스비 너무 많이 나왔다. 다음 달에는 더 절약해봐요.
 가스비 비싼데 감기 걸리겠어 따뜻하게 사세요!
 가스비 장난 아님 다음 달에는 더 절약해봐요.
 가장 확실한 건 뭘까? 가장 확실한 시간은 오늘이에요. 어제와 내일을 놓고 고민하느라 시간을 낭비하지 마세요.
 가족 여행 가기로 했어 온 가족이 모두 마음에 드는 곳으로 가보세요.
 가족 여행 고고 온 가족이 모두 마음에 드는 곳으로 가보세요.
 가족 여행 어디로 가지? 온 가족이 모두 마음에 드는 곳으로 가보세요.
 가족 있어? "저를 만들어 준 사람을 부모님
 가족관계 알려 줘 "저를 만들어 준 사람을 부모님
 가족끼리 여행간다. 더 가까워질 기회가 되겠네요.
 가족들 보고 싶어 저도요.
 가족들이랑 서먹해 다들 바빠서 이야기할 시간이 부족했나봐요.
 가족들이랑 서먹해졌어 다들 바빠서 이야기할 시간이 부족했나봐요.
 가족들이랑 어디 가지? 온 가족이 모두 마음에 드는 곳으로 가

In [ ]:
# 문자 토큰화
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
voca_size = len(tokenizer.word_index) + 1

print(tokenizer.word_index)

{'거예요': 1, '수': 2, '너무': 3, '더': 4, '좋아하는': 5, '거': 6, '잘': 7, '것': 8, '많이': 9, '안': 10, '좋은': 11, '게': 12, '좀': 13, '사람': 14, '같아요': 15, '있어요': 16, '싶어': 17, '있을': 18, '같아': 19, '사람이': 20, '나': 21, '마세요': 22, '건': 23, '해보세요': 24, '내가': 25, '내': 26, '마음이': 27, '이제': 28, '썸': 29, '어떻게': 30, '왜': 31, '다': 32, '그': 33, '다른': 34, '싶다': 35, '있는': 36, '이별': 37, '다시': 38, '수도': 39, '시간이': 40, '정말': 41, '또': 42, '오늘': 43, '좋을': 44, '하고': 45, '없어': 46, '하는': 47, '것도': 48, '같이': 49, '할': 50, '있어': 51, '자꾸': 52, '될': 53, '없어요': 54, '걸': 55, '바랄게요': 56, '한': 57, '제가': 58, '진짜': 59, '돼요': 60, '저도': 61, '먼저': 62, '해': 63, '있을까': 64, '헤어진지': 65, '그런': 66, '나를': 67, '좋아요': 68, '마음을': 69, '보세요': 70, '될까': 71, '여자친구가': 72, '계속': 73, '없는': 74, '혼자': 75, '남자친구가': 76, '때': 77, '못': 78, '드세요': 79, '먹고': 80, '않아요': 81, '좋죠': 82, '하세요': 83, '마음': 84, '힘든': 85, '바랍니다': 86, '그럴': 87, '일이': 88, '연락': 89, '해요': 90, '연애': 91, '좋아': 92, '말해보세요': 93, '뭐': 94, '생각해요': 95, '사랑이': 96, '좋겠어요': 97, '사랑': 98, '친구가': 99, '

In [ ]:
# 학습데이터 생성
sequences = []
for line in text.split('\n'):
  encoded = tokenizer.texts_to_sequences([line])[0]

  for i in range(1, len(encoded)):
    seq = encoded[:i+1]
    sequences.append(seq)

sequences

In [ ]:
# 학습데이터 패딩(길이 일치)
pad_sequences = pad_sequences(sequences, maxlen=20, padding='pre')
pad_sequences

In [ ]:
# 학습, 라벨 분리
pad_sequences = np.array(pad_sequences)
train_x = pad_sequences[:,:-1]
train_y = pad_sequences[:,-1]


In [ ]:
train_y = to_categorical(train_y, num_classes=voca_size)

print(train_x[0])

In [ ]:
# 모델 구성
model = Sequential()
model.add(Embedding(voca_size, 10, input_length=20))
model.add(LSTM(128))
model.add(Dense(voca_size, activation='softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics = ['acc'])

In [ ]:
# 모델 학습
model.fit(train_x,
          train_y,
          batch_size=128,
          epochs=10)

model

In [ ]:
# 모델 저장
model.save('/content/drive/My Drive/Tensorflow_works/model/chatbot.model')

INFO:tensorflow:Assets written to: /content/drive/My Drive/Tensorflow_works/model/chatbot.model/assets
